In [10]:
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.sparse.linalg import svds
from scipy.sparse import csc_matrix
import re

# Methods to calculate svd
from numpy.linalg import svd

In [11]:
from functools import partial
import concurrent.futures as cf

try:
    from joblib import Parallel, delayed 
    _HAVE_JOBLIB = True
except Exception:
    _HAVE_JOBLIB = False

# Computes the full SVD of matrix A and returns the top k singular values.
def truncated_svd(A, k=None, include_u_v=False):
    S = svd(A, compute_uv=False, full_matrices=False)
    if k is None:
        k = len(S)
    S_k = S[:k]
    if include_u_v:
        U, _, Vh = svd(A, full_matrices=False)
        return S_k, U[:, :k], Vh[:k, :]
    return S_k


# Computes the log-likelihood for a given q
def compute_ll_for_q(sv: np.ndarray, p: int, q: int):
    q = int(q)
    S1 = sv[:q]
    S2 = sv[q:]

    if len(S2) == 0 or p <= 2:
        return (q, float(-np.inf))

    mu1 = np.mean(S1) if len(S1) else 0.0
    mu2 = np.mean(S2) if len(S2) else 0.0

    if len(S1) > 1:
        s1_squared = np.var(S1, ddof=1)
    else:
        s1_squared = 0.0

    if len(S2) > 1:
        s2_squared = np.var(S2, ddof=1)
    else:
        s2_squared = 0.0

    sigma2 = ((q - 1) * s1_squared + (p - q - 1) * s2_squared) / (p - 2)
    if not np.isfinite(sigma2) or sigma2 <= 0:
        return (q, float(-np.inf))

    sigma = np.sqrt(sigma2)

    ll = 0.0
    if len(S1):
        ll += float(np.sum(norm.logpdf(S1, mu1, sigma)))
    if len(S2):
        ll += float(np.sum(norm.logpdf(S2, mu2, sigma)))

    return (q, ll)

# Estimate the embedding dimension using the profile likelihood method
def embedding_dimension(singular_values, k=None, n_jobs: int = -1):
    
    if k is None:
        k = len(singular_values)

    sv = np.array(singular_values[:k], dtype=float)
    p = len(sv)

    if p <= 2:
        return 1

    qs = list(range(1, p))

    if n_jobs == 1:
        best_ll = float(-np.inf)
        d_hat = 1
        for q in qs:
            _, ll = compute_ll_for_q(sv, p, q)
            if ll > best_ll:
                best_ll = ll
                d_hat = q
        return d_hat

    if _HAVE_JOBLIB:
        results = Parallel(n_jobs=n_jobs, backend="loky")(
            delayed(compute_ll_for_q)(sv, p, q) for q in qs
        )
    else:
        max_workers = None if n_jobs in (None, -1) else int(n_jobs)
        func = partial(compute_ll_for_q, sv, p)
        with cf.ProcessPoolExecutor(max_workers=max_workers) as ex:
            results = list(ex.map(func, qs))

    if not results:
        return 1

    best_q, _best_ll = max(results, key=lambda t: t[1]) 
    return int(best_q)

In [ ]:
df_2020 = pd.read_csv('../data/Vaccines/EdgesSource_Target_W_year=2020.csv')
df_2021 = pd.read_csv('../data/Vaccines/EdgesSource_Target_W_year=2021.csv')
df_2022 = pd.read_csv('../data/Vaccines/EdgesSource_Target_W_year=2022.csv')

print("CSV files loaded successfully.")

# Print number of edges in each dataset
print("DATA COUNT BY YEAR")
print(f"\nYear 2020:")
print(f"  - Number of edges: {len(df_2020)}")
print(f"  - Columns: {list(df_2020.columns)}")

print(f"\nYear 2021:")
print(f"  - Number of edges: {len(df_2021)}")
print(f"  - Columns: {list(df_2021.columns)}")

print(f"\nYear 2022:")
print(f"  - Number of edges: {len(df_2022)}")
print(f"  - Columns: {list(df_2022.columns)}")
print("Setting weights to 1...")

if 'Weight' in df_2020.columns:
    df_2020['Weight'] = 1
if 'Weight' in df_2021.columns:
    df_2021['Weight'] = 1
if 'Weight' in df_2022.columns:
    df_2022['Weight'] = 1
    
G_2020 = nx.from_pandas_edgelist(df_2020, source='anonyme_Source', target='anonyme_Target', edge_attr='Weight', create_using=nx.Graph())
G_2021 = nx.from_pandas_edgelist(df_2021, source='anonyme_Source', target='anonyme_Target', edge_attr='Weight', create_using=nx.Graph())
G_2022 = nx.from_pandas_edgelist(df_2022, source='anonyme_Source', target='anonyme_Target', edge_attr='Weight', create_using=nx.Graph())

print("\nGraphs successfully generated with NetworkX.")
print(f"Graph 2020 - Number of nodes: {G_2020.number_of_nodes()}, Number of edges: {G_2020.number_of_edges()}")
print(f"Graph 2021 - Number of nodes: {G_2021.number_of_nodes()}, Number of edges: {G_2021.number_of_edges()}")
print(f"Graph 2022 - Number of nodes: {G_2022.number_of_nodes()}, Number of edges: {G_2022.number_of_edges()}")

CSV files loaded successfully.
DATA COUNT BY YEAR

Year 2020:
  - Number of edges: 223099
  - Columns: ['anonyme_Source', 'anonyme_Target', 'Weight']

Year 2021:
  - Number of edges: 75750
  - Columns: ['anonyme_Source', 'anonyme_Target', 'Weight']

Year 2022:
  - Number of edges: 14604
  - Columns: ['anonyme_Source', 'anonyme_Target', 'Weight']
Setting weights to 1...

Graphs successfully generated with NetworkX.
Graph 2020 - Number of nodes: 113027, Number of edges: 222566
Graph 2021 - Number of nodes: 18826, Number of edges: 40394
Graph 2022 - Number of nodes: 3617, Number of edges: 7910


In [ ]:
metrics = {}

for year, G in zip([2021, 2022], [G_2021, G_2022]):
    print(f"\n{year}...")

    communities = nx.community.louvain_communities(G)
    partition = {node: idx for idx, community in enumerate(communities) for node in community}
    nx.set_node_attributes(G, partition, "community")

    modularity = nx.community.modularity(G, communities)
    assortativity = nx.attribute_assortativity_coefficient(G, "community")
    density = nx.density(G)
    clustering_coefficient = nx.average_clustering(G)

    print(f"Comunities: {len(communities)}")
    print(f"Modularity: {modularity}")
    print(f"Assortativity: {assortativity}")
    print(f"Density: {density}")
    print(f"Clustering Coefficient: {clustering_coefficient}")

    if year == 2020:
        k = 8000
        A = nx.to_scipy_sparse_array(G, format='csr', dtype=float)
        u, s, vt = svds(A, k=k, which='LM')
        singular_values = sorted(s, reverse=True)
        d_hat = embedding_dimension(singular_values)
        metrics[year] = {
            "singular_values": singular_values,
            "embedding_dimension": d_hat,
            "modularity": modularity,
            "assortativity": assortativity,
            "density": density,
            "clustering_coefficient": clustering_coefficient
        }
        print(f"Using sparse SVD for year {year} with k={k}")
        print(f"Embedding Dimension {year}: {d_hat}")
    else:
        A = nx.to_numpy_array(G, dtype=int )
        singular_values = truncated_svd(A)
        d_hat = embedding_dimension(singular_values)
        metrics[year] = {
            "singular_values": singular_values,
            "embedding_dimension": d_hat,
            "modularity": modularity,
            "assortativity": assortativity,
            "density": density,
            "clustering_coefficient": clustering_coefficient
        }
        print(f"Embedding Dimension {year}: {d_hat}")
    



2021...
Comunities: 159
Modularity: 0.8898859343845554
Assortativity: 0.9661178079065924
Density: 0.00022795746438094078
Clustering Coefficient: 0.08671400822456832
Embedding Dimension 2021: 2928

2022...
Comunities: 71
Modularity: 0.8115474419072979
Assortativity: 0.9433629800904899
Density: 0.0012095659386231685
Clustering Coefficient: 0.09014308488788474
Embedding Dimension 2022: 658
